<a href="https://colab.research.google.com/github/OmdenaAI/omdena-colombia-socialnetwork/blob/task-1c-reddit-data-extraction/src/tasks/task-1-data-scraping/task-1c-reddit-data-extraction/Interactive_reddit_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and install useful libraries

In [1]:
!pip install praw -q
!pip install psaw -q

     |████████████████████████████████| 167 kB 5.6 MB/s 
     |████████████████████████████████| 52 kB 848 kB/s 


In [2]:
import praw
import pandas as pd
from psaw import PushshiftAPI
import datetime 
from google.colab import files
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
!mkdir CSV_FILES   # folder for saving the csv
%cd /content/CSV_FILES

/content/CSV_FILES


# Authentication
Follow this article https://towardsdatascience.com/scraping-reddit-data-1c0af3040768 to get your client id, secret and agent details.

In [4]:
# Authenticate
my_client_id = '' #@param {type: "string"}
my_client_secret = '' #@param {type: "string"}
my_user_agent = '' #@param {type: "string"}
api = PushshiftAPI()
reddit = praw.Reddit(client_id=my_client_id, client_secret=my_client_secret, user_agent=my_user_agent, check_for_async=False)

# Extract comments by URL

In [5]:
# Function to scrape data from reddit and save it as a dataframe
def reddit_scraper(urls):
  posts_url = []
  for url in urls:
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
      posts_url.append(comment.body)
  return posts_url

In [14]:
urls = ['https://www.reddit.com/r/Colombia/comments/n40s29/megathread_de_protestas_2021/', 
        'https://www.reddit.com/r/Colombia/comments/jrtyn0/para_ls_libertatis_de_colombia_consideran_que/', 
        'https://www.reddit.com/r/Colombia/comments/n2vzqr/colombia_est%C3%A1_pasando_por_un_momento_grav%C3%ADsimo_y/', 
        'https://www.reddit.com/r/Colombia/comments/n6bl52/quiero_felicitar_a_todos_los_colombianos_y/', 
        'https://www.reddit.com/r/Colombia/comments/n2vzqr/colombia_est%C3%A1_pasando_por_un_momento_grav%C3%ADsimo_y/', 
        'https://www.reddit.com/r/Colombia/comments/nfqunl/cu%C3%A1l_es_la_opinion_de_ustedes_sobre_los_bloqueos/', 
        'https://www.reddit.com/r/Colombia/comments/nbtnqc/%C3%A1lvaro_uribe_violencia_en_colombia_busca_tumbar/', 
        'https://www.reddit.com/r/Colombia/comments/owtv86/por_que_seguimos_perdiendo_el_tiempo_entre/']

df = pd.DataFrame(reddit_scraper(urls),columns=["body"])
df

,body
0,"Hola Amigos. I am a 25 year old born and raised in Canada to first generation Colombian parents. I speak fluent Spanish but I feel like I can express myself better in english. I was in Colombia for the first time 2 years ago on a solo 4 month backpacking trip and it was the best 4 months of my life. During that time I saw and experienced more of Colombia than my parents ever have. Colombians are the nicest people I have ever met. I could go on for days with stories of kind strangers. Everyone I met treated me like i was part of their family. It was the first time I was surrounded with my own people. Growing up in a suburban dominantly white neighbourhood, I had never experienced that. It was a very life changing experience. It made me so proud to be where I am from. The Colombian people's passion and patriotism is what really stood out for me. I am so proud of my brothers and sister for standing up to what is right. Seeing the news makes me so sad. I have been crying. Looking at videos of police brutality against civilians makes me tear up every time. It makes me very angry. I have lived an incredibly privileged life and feel like I shouldn't be getting so emotional over something that is not directly affecting my life here in Canada. I feel like an impostor or something. I just want to say I am sending my support and I wish I could help in anyway. Viva Colombia ❤️"
1,"Vivo en Tabio, un pueblo una hora al norte de Bogotá, las protestas son pacíficas pero con todo el perrenque... Soy nuevo en Reddit además y no sé cómo adjuntar una foto xdd"
2,[deleted]
3,Fuerza desde Argentina hermanos. \n\n\nDuque y Uribe estan cometiendo delitos de lesa humanidad.
4,Acabo de revisar Twitter y las etiquetas uribistas han sido inundadas de fancams. El activismo social de los fanáticos de K-Pop me divierte y me confunde en igual medida.
...,...
212,Criptomonedas!y NUNCA tratarse con Cámara de Comercio NUNCA!
213,"Si la deforestación para siembra y vías en lugares cada vez más lejanos no es un daño ecológico o si el uso de glifosato para acabar terrenos cultivados (en pausa) o los vertimientos accidentales de químicos en laboratorios de producción no son un daño ecológico, me temo que le falta embarrarse las botas después de andar por trochas en área rural de Colombia."
214,Le pido que de cifras especificas por favor
215,Es fácil desvirtuar con comodidades en vez de tener una conversación como mencionó el OP de alguien que en diferentes regiones ha aprendido un par de lecciones que van más allá del bien y el mal.


# Extract comments by keywords

In [15]:
# Function to get the comments from subreddits results
start_date = '2021-04-01'  #@param {type: "date"}
end_date = '2021-08-27'  #@param {type: "date"}
limit =   100#@param {type: "number"}

year_1, month_1, day_1 = [int(x) for x in start_date.split("-")]
year_2, month_2, day_2 = [int(x) for x in end_date.split("-")]

start_time = int(datetime.datetime(year_1,month_1,day_1).timestamp())
end_time = int(datetime.datetime(year_2,month_2,day_2).timestamp())

def get_comments(subreddit, keyword):
  posts = []
  # Get the URLs
  submissions = list(api.search_submissions(after= start_time, before=end_time,
                             subreddit = subreddit,
                             q = keyword,
                             filter = ['full_link', 'num_comments', ],
                             limit = limit))
  sub = pd.DataFrame(submissions)
  comments = sub[sub['num_comments']>0] #only post with comments
  comments.astype('str')
  urls = comments['full_link'].tolist()
# Get the comments
  for url in urls:
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
      posts.append(comment.body)
  return posts


In [16]:
keywords = ["protest"]
# Creating dataframes for differents subreddits and keywords
dfs = [pd.DataFrame(get_comments('Colombia', keyword),columns=["body"]) for keyword in keywords]
dfs.append(df)
# Data as a single dataframe
reddit_data = pd.concat(dfs, ignore_index=True)
reddit_data

,body
0,Que Lamparas
1,"Y no esta muerto hasta donde se, solo estaba convulsiónando"
2,Cali?
3,tipico comunista dando pena
4,Contexto? Que paso?
...,...
393,Criptomonedas!y NUNCA tratarse con Cámara de Comercio NUNCA!
394,"Si la deforestación para siembra y vías en lugares cada vez más lejanos no es un daño ecológico o si el uso de glifosato para acabar terrenos cultivados (en pausa) o los vertimientos accidentales de químicos en laboratorios de producción no son un daño ecológico, me temo que le falta embarrarse las botas después de andar por trochas en área rural de Colombia."
395,Le pido que de cifras especificas por favor
396,Es fácil desvirtuar con comodidades en vez de tener una conversación como mencionó el OP de alguien que en diferentes regiones ha aprendido un par de lecciones que van más allá del bien y el mal.


In [10]:
# Save as csv
reddit_data.to_csv('reddit_data_2.csv')
files.download('reddit_data_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>